**Maestría en Informática - PUCP - Aprendizaje Automático**

**Proyecto Grupal 2022-1**

# **Complicaciones en pacientes que sufrieron infarto al miocardio**

Repositorio: https://github.com/dp202204/Proyecto-ML-2022

In [75]:
import pandas as pd

In [76]:
# Carga de datos - Pacientes que sufrieron infarto al miocardio y sus complicaciones
# Datos utilizados:
# Myocardial infarction complications Data Set
# Center for Machine Learning and Intelligent Systems, Bren School of Information and Computer Science, University of California, Irvine

mic_data1 = pd.read_csv('https://leicester.figshare.com/ndownloader/files/23581310')

In [77]:
mic_data1.shape

(1700, 124)

In [78]:
# Recolección de información general acerca de las columnas y su contenido

cant_col_nan      = 0
cant_col_sin_nan  = 0
cant_col_num      = 0
cant_col_binarias = 0

for col in mic_data1:
  #Determinamos cuantas columna son numericas
  if pd.api.types.is_numeric_dtype(mic_data1[col]):
    cant_col_num = cant_col_num + 1
    if mic_data1[col].nunique() == 2:
      cant_col_binarias = cant_col_binarias + 1
  #Determinamos que columnas tienen datos faltantes
  if mic_data1[col].isna().sum() > 0:
    cant_col_nan = cant_col_nan + 1
  else:
     cant_col_sin_nan = cant_col_sin_nan + 1

In [79]:
cant_col_mic_data1 = mic_data1.shape[1]
print('Total de columnas en el conjunto de datos: ',cant_col_mic_data1)
print('Total de columnas numericas:               ',cant_col_num, ' (', '%.1f%%' % (100*cant_col_num/cant_col_mic_data1),')')
print('Total de columnas numericas binarias:       ',cant_col_binarias, ' (', '%.1f%%' % (100*cant_col_binarias/cant_col_mic_data1),')')
print('Total de columnas con datos faltantes:     ',cant_col_nan, ' (', '%.1f%%' % (100*cant_col_nan/cant_col_mic_data1),')')
print('Total de columnas con datos completos:      ',cant_col_sin_nan, ' (Para comprobación)')

Total de columnas en el conjunto de datos:  124
Total de columnas numericas:                124  ( 100.0% )
Total de columnas numericas binarias:        89  ( 71.8% )
Total de columnas con datos faltantes:      110  ( 88.7% )
Total de columnas con datos completos:       14  (Para comprobación)


Todas las columnas son numéricas. Algunas son enteras y otras de punto flotante.

89 columnas (71.8%) son binarias, es decir, tiene los valores 0 y 1. Por tanto 35 columan tiene datos no binarios, de estas una parte son ordinales y las identificaremos una a una.

La mayoría de las columnas tienen datos faltantes (88.7%).

In [80]:
# Recoleccion de características más específicas de los datos por columna: Datos unicos, datos faltantes

cant_filas_mic_data1 = mic_data1.shape[0]
mic_caract = pd.DataFrame(mic_data1.nunique(),columns=['Datos_Unicos'])
mic_caract['Faltantes'] = mic_data1.isna().sum() / cant_filas_mic_data1 * 100
# mic_data.nunique().hist()
# mic_data.nunique()

In [81]:
print ('Columnas con mayor cantidad de datos únicos')
print ()
mic_caract.sort_values('Datos_Unicos', ascending = False)['Datos_Unicos'].head(37)

Columnas con mayor cantidad de datos únicos



ID             1700
L_BLOOD         174
ALT_BLOOD        69
AGE              62
ROE              58
AST_BLOOD        58
K_BLOOD          51
NA_BLOOD         40
S_AD_ORIT        32
S_AD_KBRIG       30
D_AD_KBRIG       21
D_AD_ORIT        20
TIME_B_S          9
LET_IS            8
DLIT_AG           8
STENOK_AN         7
post_im           5
inf_im            5
lat_im            5
ant_im            5
NA_R_1_n          5
NOT_NA_1_n        5
ZSN_A             5
FK_STENOK         5
KFK_BLOOD         4
R_AB_2_n          4
R_AB_1_n          4
NA_R_2_n          4
NOT_NA_2_n        4
INF_ANAM          4
R_AB_3_n          4
GB                4
NOT_NA_3_n        3
NA_R_3_n          3
IBS_POST          3
NA_KB             2
fibr_ter_06       2
Name: Datos_Unicos, dtype: int64

De acuerdo al análisis de datos únicos, solo la columna ID contiene información que no sería útil.

Descartaremos la columna ID.

In [82]:
print ('Columnas con mayor porcentaje de datos faltantes')
print ()
mic_caract.sort_values('Faltantes', ascending = False)['Faltantes'].head(15)

Columnas con mayor porcentaje de datos faltantes



KFK_BLOOD     99.764706
IBS_NASL      95.764706
S_AD_KBRIG    63.294118
D_AD_KBRIG    63.294118
NOT_NA_KB     40.352941
LID_KB        39.823529
NA_KB         38.647059
GIPER_NA      22.058824
NA_BLOOD      22.058824
K_BLOOD       21.823529
GIPO_K        21.705882
AST_BLOOD     16.764706
ALT_BLOOD     16.705882
S_AD_ORIT     15.705882
D_AD_ORIT     15.705882
Name: Faltantes, dtype: float64

De acuerdo a algunos autores, la proporción de valores faltantes en estudios científicos es muy variable y puede alcanzar niveles de entre 26% to 72% para estudios sobre educación y psicología, siendo valores comunes entre 15% to 20%.

(Yiran Dong, Chao-Ying Joanne Peng, Principled missing data methods for researchers, Springerplus. 2013; 2: 222. Published online 2013 May 14. doi: 10.1186/2193-1801-2-222
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3701793/ )

Notas diversas encontradas en foros de internet sugieren valores entre 25% y 30% como aceptables.

Para el presente trabajo descartaremos las columnas con más de 25% de datos faltantes.

Descartaremos 7 columnas, que serán las siguientes:

- KFK_BLOOD (Serum CPK content). Faltantes: 99.8%

- IBS_NASL (Heredity on CHD)). Faltantes: 95.8%

- S_AD_KBRIG (Systolic blood pressure according to Emergency Cardiology Team). Faltantes: 63.3%

- D_AD_KBRIG (Diastolic blood pressure according to Emergency Cardiology Team). Faltantes: 63.3%

- NOT_NA_KB (Use of NSAIDs by the Emergency Cardiology Team). Faltantes: 40.4%

- LID_KB (Use of lidocaine by the Emergency Cardiology Team). Faltantes: 39.8%

- NA_KB (Use of opioid drugs by the Emergency Cardiology Team). Faltantes: 38.6%

Nota: Todas estas columnas corresponden a datos médicos de los pacientes durante su tratamiento. No contienen información sobre las complicaciones.

In [83]:
# PREPROCESAMIENTO - PASO 1
# Eliminación de columnas con muchos datos únicos y faltantes.
# Se eliminan 8 columnas en total.

mic_data2 = mic_data1.copy()

col_elim = ['ID','KFK_BLOOD','IBS_NASL','S_AD_KBRIG','D_AD_KBRIG','NOT_NA_KB','LID_KB','NA_KB']
mic_data2.drop(col_elim, axis=1, inplace=True)

In [84]:
mic_data2.shape

(1700, 116)

De acuerdo a la descripción de datos, las últimnas 12 columnas tienen informacion sobre las complicaciones.
- Columnas 113 a 123 (número original de columna): Complicaciones específicas. Todas tienen valores binarios.
- Columna 124 (número original de columna) - LET_IS: Causa de resultado letal. Contiene un número: Cero significa que el caso no fue letal. Del 1 al 7 indica la causa de la letalidad.

Analizaremos qué proporción de casos presentaron complicaciones, cuántas complicaciones se pueden presentar en cada caso y qué proporción de casos fueron letales.

In [85]:
col_causas = ['FIBR_PREDS','PREDS_TAH','JELUD_TAH','FIBR_JELUD','A_V_BLOK','OTEK_LANC','RAZRIV','DRESSLER','ZSN','REC_IM','P_IM_STEN']

tuvo_complic = mic_data2[col_causas].sum(axis=1) > 0

# mic_data2['CON_COMPLIC'] = mic_data2[tuvo_complic]
# mic_data2['LETAL'] = mic_data2['LET_IS'] > 0
mic_data2['NUM_COMPLIC'] = mic_data2[col_causas].sum(axis=1)

print('Casos ordenados por número de complicaciones:')
mic_data2.sort_values('NUM_COMPLIC', ascending = False)['NUM_COMPLIC'].head(15)

Casos ordenados por número de complicaciones:


1121    5
1376    5
1163    4
1624    4
1489    4
1374    4
1288    4
1197    4
1194    4
894     4
1226    4
1178    4
1005    4
1022    4
1258    4
Name: NUM_COMPLIC, dtype: int64

**Se aprecia que puede haber casos hasta con 5 complicaciones.**

**Por lo tanto, el modelo predictivo no podría usar las columnas de complicaciones como categoría de clasificación porque no son excluyentes.**

In [86]:
seleccion_casos_complic = mic_data2['NUM_COMPLIC'] > 0
num_casos_complic = len(mic_data2.loc[mic_data2['NUM_COMPLIC'] > 0])

num_casos_let = len(mic_data2.loc[mic_data2['LET_IS'] > 0])

cant_casos_mic_data2 = mic_data2.shape[0]
print('Total de casos en el conjunto de datos: ',cant_casos_mic_data2)
print('Total de casos con complicaciones:      ',num_casos_complic, ' (', '%.1f%%' % (100*num_casos_complic/cant_casos_mic_data2),')')
print('Total de casos letales:                 ',num_casos_let, ' (', '%.1f%%' % (100*num_casos_let/cant_casos_mic_data2),')')


Total de casos en el conjunto de datos:  1700
Total de casos con complicaciones:       935  ( 55.0% )
Total de casos letales:                  271  ( 15.9% )


# El modelo podría enfoarse en predecir si habrá complicaciones en general o si habrá complicacaiones letales.

# De acuerdo a la literatura revisada, las complicaciones son muy comunes (se comprueba con el 55% observado en la en conjunto de datos) y que éstas pueden ser muy leves.

# Para efectos del presente trabajo, nos concentramremos en predecir si puede haber complicaciones letales, que son los casos más importantes.

## Preprocesamiento: Reemplazo de valores nulos

In [87]:
# Realizando una copia del data frame
mic_data3 = mic_data2.copy()

### Columnas Binarias

In [88]:
# Recolección de información general acerca de las columnas y su contenido

col_binary_name = []

for col in mic_data3:
  #Determinamos cuantas columna son numericas
    if mic_data3[col].nunique() == 2:
      col_binary_name.append(col)
      
col_binary_name

['SEX',
 'SIM_GIPERT',
 'nr_11',
 'nr_01',
 'nr_02',
 'nr_03',
 'nr_04',
 'nr_07',
 'nr_08',
 'np_01',
 'np_04',
 'np_05',
 'np_07',
 'np_08',
 'np_09',
 'np_10',
 'endocr_01',
 'endocr_02',
 'endocr_03',
 'zab_leg_01',
 'zab_leg_02',
 'zab_leg_03',
 'zab_leg_04',
 'zab_leg_06',
 'O_L_POST',
 'K_SH_POST',
 'MP_TP_POST',
 'SVT_POST',
 'GT_POST',
 'FIB_G_POST',
 'IM_PG_P',
 'ritm_ecg_p_01',
 'ritm_ecg_p_02',
 'ritm_ecg_p_04',
 'ritm_ecg_p_06',
 'ritm_ecg_p_07',
 'ritm_ecg_p_08',
 'n_r_ecg_p_01',
 'n_r_ecg_p_02',
 'n_r_ecg_p_03',
 'n_r_ecg_p_04',
 'n_r_ecg_p_05',
 'n_r_ecg_p_06',
 'n_r_ecg_p_08',
 'n_r_ecg_p_09',
 'n_r_ecg_p_10',
 'n_p_ecg_p_01',
 'n_p_ecg_p_03',
 'n_p_ecg_p_04',
 'n_p_ecg_p_05',
 'n_p_ecg_p_06',
 'n_p_ecg_p_07',
 'n_p_ecg_p_08',
 'n_p_ecg_p_09',
 'n_p_ecg_p_10',
 'n_p_ecg_p_11',
 'n_p_ecg_p_12',
 'fibr_ter_01',
 'fibr_ter_02',
 'fibr_ter_03',
 'fibr_ter_05',
 'fibr_ter_06',
 'fibr_ter_07',
 'fibr_ter_08',
 'GIPO_K',
 'GIPER_NA',
 'NITR_S',
 'LID_S_n',
 'B_BLOK_S_n',
 'AN

In [89]:
len(col_binary_name)

85

In [90]:
# Creando procedimiento
for col in col_binary_name:
  # encontrar la moda de la columna
  moda = mic_data3[col].mode()[0]

  # guardar los nulos
  Col_NaN = mic_data3[mic_data3[col].isnull()].index

  # Imputamos la moda
  mic_data3.loc[Col_NaN,col] = moda


In [91]:
# Verificando si existen nulos en la lista de columnas binarias
cant_col_nan = 0

for col in col_binary_name:
  # Verificar nulos
  if mic_data3[col].isna().sum() > 0:
    cant_col_nan = cant_col_nan + 1

cant_col_nan

0

### Ordinarias

In [92]:
col_ordinal_name = {'INF_ANAM','STENOK_AN','FK_STENOK','IBS_POST','GB','DLIT_AG','ZSN_A','ant_im','lat_im','inf_im','post_im','TIME_B_S','R_AB_1_n','NA_R_1_n','NOT_NA_1_n','R_AB_2_n','NA_R_2_n','NOT_NA_2_n','R_AB_3_n','NA_R_3_n','NOT_NA_3_n'}


In [93]:
# Creando procedimiento
for col in col_ordinal_name:
  # encontrar la moda de la columna
  moda = mic_data3[col].mode()[0]

  # guardar los nulos
  Col_NaN = mic_data3[mic_data3[col].isnull()].index

  # Imputamos la moda
  mic_data3.loc[Col_NaN,col] = moda

In [94]:
# Verificando si existen nulos en la lista de columnas binarias
cant_col_nan2 = 0

for col in col_ordinal_name:
  # Verificar nulos
  if mic_data3[col].isna().sum() > 0:
    cant_col_nan2 = cant_col_nan2 + 1

cant_col_nan2

0

### Numericas

In [95]:
# Creando procedimiento
for col in mic_data3:
  # encontrar la moda de la columna
  mediana = mic_data3[col].median()

  # Imputar la mediana a los nulos
  mic_data3[col].fillna(mediana, inplace=True)

In [96]:
# Verificando si existen nulos en la lista de columnas binarias
cant_col_nan3 = 0

for col in mic_data3:
  # Verificar nulos
  if mic_data3[col].isna().sum() > 0:
    cant_col_nan3 = cant_col_nan3 + 1

cant_col_nan3

0

## Crear la variable explicada (y)


*   Creamos una columna 



In [100]:
# Realizando una copia del data frame
mic_data4 = mic_data3.copy()

In [101]:
# Verificamos los valores de la columna 'LET_IS'
mic_data4['LET_IS'].value_counts()

0    1429
1     110
3      54
7      27
6      27
4      23
2      18
5      12
Name: LET_IS, dtype: int64

In [102]:
# Diccionario para crear la columna target
Diccionario = {0:0, 1:1, 2:1, 3:1, 4:1, 5:1, 6:1, 7:1}

# Columna target
mic_data4['LETALIDAD'] = mic_data4['LET_IS']

# Ajuste columna target
mic_data4=mic_data4.replace({ 'LETALIDAD': Diccionario})
  

In [103]:
# Verificamos los valores de la columna 'LETALIDAD'
mic_data4['LETALIDAD'].value_counts()

0    1429
1     271
Name: LETALIDAD, dtype: int64

In [ ]:
# Borrar las columnas
col_a_eliminar = ['FIBR_PREDS','PREDS_TAH','JELUD_TAH','FIBR_JELUD','A_V_BLOK','OTEK_LANC','RAZRIV','DRESSLER','ZSN','REC_IM','P_IM_STEN', 'LET_IS']
mic_data4.drop(col_a_eliminar, axis=1, inplace=True)

In [106]:
mic_data4.shape

(1700, 106)

## Normalizando los datos

In [107]:
# Obteniendo la lista de columnas no binarias, no ordinales a normalizar
columnas_a_normalizar = []

for col in mic_data4:
  if mic_data4[col].nunique() != 2:
    columnas_a_normalizar.append(col)

# Visualizar las columnas encontradas
columnas_a_normalizar

['AGE',
 'INF_ANAM',
 'STENOK_AN',
 'FK_STENOK',
 'IBS_POST',
 'GB',
 'DLIT_AG',
 'ZSN_A',
 'S_AD_ORIT',
 'D_AD_ORIT',
 'ant_im',
 'lat_im',
 'inf_im',
 'post_im',
 'K_BLOOD',
 'NA_BLOOD',
 'ALT_BLOOD',
 'AST_BLOOD',
 'L_BLOOD',
 'ROE',
 'TIME_B_S',
 'R_AB_1_n',
 'R_AB_2_n',
 'R_AB_3_n',
 'NA_R_1_n',
 'NA_R_2_n',
 'NA_R_3_n',
 'NOT_NA_1_n',
 'NOT_NA_2_n',
 'NOT_NA_3_n',
 'NUM_COMPLIC']

In [108]:
# Normalizar los valores entre 0 y 1
from sklearn.preprocessing import MinMaxScaler

estandarizador = MinMaxScaler()
mic_data4[columnas_a_normalizar] = estandarizador.fit_transform(mic_data4[columnas_a_normalizar])
mic_data4.head()

,AGE,SEX,INF_ANAM,STENOK_AN,FK_STENOK,IBS_POST,GB,SIM_GIPERT,DLIT_AG,ZSN_A,...,NOT_NA_3_n,LID_S_n,B_BLOK_S_n,ANT_CA_S_n,GEPAR_S_n,ASP_S_n,TIKL_S_n,TRENT_S_n,NUM_COMPLIC,LETALIDAD
0,0.772727,1,0.666667,0.166667,0.25,1.0,1.000000,0.0,1.000000,0.00,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0
1,0.439394,1,0.333333,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.00,...,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0
2,0.393939,1,0.000000,0.000000,0.00,1.0,0.666667,0.0,0.285714,0.00,...,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0
3,0.636364,0,0.000000,0.000000,0.00,1.0,0.666667,0.0,0.428571,0.25,...,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.2,0
4,0.515152,1,0.000000,0.000000,0.00,1.0,1.000000,0.0,1.000000,0.00,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0


In [109]:
# Verificando que los valores se encuentren entre 0 y 1 (min y max respectivamente)
mic_data4.describe()

,AGE,SEX,INF_ANAM,STENOK_AN,FK_STENOK,IBS_POST,GB,SIM_GIPERT,DLIT_AG,ZSN_A,...,NOT_NA_3_n,LID_S_n,B_BLOK_S_n,ANT_CA_S_n,GEPAR_S_n,ASP_S_n,TIKL_S_n,TRENT_S_n,NUM_COMPLIC,LETALIDAD
count,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,...,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000
mean,0.543369,0.626471,0.184510,0.361961,0.309853,0.592941,0.465490,0.033529,0.407563,0.047059,...,0.039118,0.281765,0.126471,0.669412,0.717647,0.746471,0.017647,0.200588,0.158706,0.159412
std,0.170207,0.483883,0.278749,0.404786,0.257721,0.401084,0.362270,0.180067,0.442421,0.162266,...,0.170945,0.449992,0.332477,0.470563,0.450277,0.435159,0.131704,0.400558,0.181065,0.366167
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.424242,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.560606,1.000000,0.000000,0.166667,0.500000,0.500000,0.666667,0.000000,0.142857,0.000000,...,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.200000,0.000000
75%,0.666667,1.000000,0.333333,0.833333,0.500000,1.000000,0.666667,0.000000,1.000000,0.000000,...,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.200000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
